# Exploring industry returns in the Fama-French data-set

The [49 Industry Portfolios](https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/det_49_ind_port.html) data-set has a series of daily returns of 49 different industry types. Here, we look at some of their statistical properties.

The documentation for the Fama-French data-set can be found [here](https://plutopy.readthedocs.io/en/latest/FamaFrench.html) and [here](https://shyams80.github.io/plutoR/docs/reference/FamaFrench-class.html)

In [1]:
library(tidyverse)
library(ggthemes)
library(odbc)
library(plutoR)
library(quantmod)
library(lubridate)
library(reshape2)
library(PerformanceAnalytics)
library(ggrepel)
library(tbl2xts)

options("scipen"=999)
options(stringsAsFactors = FALSE)

source("config.R")
source("goofy/plot.common.R")

#initialize
famaFrench <- FamaFrench()

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.2.1     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   0.8.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: xts
Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric

Registered S3 method overwritten by 'xts':
  method     from
  as.zoo.xts zoo 

Attaching package: ‘xts’

The following objects are masked from ‘package:dplyr’:

    first, last

Loading required package: TTR
Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 
Version 0.4-0 included new data defaults. See ?getSymbols.

Attaching package: ‘lubridate’

The following object is masked from ‘package:b

In [2]:
startDt <- as.Date("1990-01-01")

#value (market-cap) weighted
valueWtd <- famaFrench$Industry49Daily() %>%
    filter(RET_TYPE == 'AVWRD' & TIME_STAMP >= startDt) %>%
    mutate(R = RET/100) %>%
    select(DATE = TIME_STAMP, KEY_ID, R) %>%
    collect() %>% 
    # the KEY_ID column has industry id's
    # we want them as column names
    mutate(group=1) %>%
    spread(KEY_ID, R) %>%
    select(-group) %>%
    tbl_xts()

#equal weighted
equalWtd <- famaFrench$Industry49Daily() %>%
    filter(RET_TYPE == 'AEWRD' & TIME_STAMP >= startDt) %>%
    mutate(R = RET/100) %>%
    select(DATE = TIME_STAMP, KEY_ID, R) %>%
    collect() %>% 
    # the KEY_ID column has industry id's
    # we want them as column names
    mutate(group=1) %>%
    spread(KEY_ID, R) %>%
    select(-group) %>%
    tbl_xts()

In [3]:
lookbackDays <- 220 * 5

rrValueWtd <- rollapply(valueWtd, lookbackDays, Return.cumulative, by.column = F)
rrEqualWtd <- rollapply(equalWtd, lookbackDays, Return.cumulative, by.column = F)

rrValueWtd <- na.omit(rrValueWtd)
rrEqualWtd <- na.omit(rrEqualWtd)

print(head(rrValueWtd))
print(head(rrEqualWtd))

                [,1]      [,2]      [,3]      [,4]      [,5]      [,6]
1994-05-06 0.6250045 0.1544141 0.7766170 0.8335946 0.9109250 0.6916487
1994-05-09 0.5818277 0.1326520 0.6879995 0.7928641 0.8717527 0.6428463
1994-05-10 0.5650564 0.1511583 0.7217697 0.8096282 0.8999900 0.6397283
1994-05-11 0.5462040 0.1318101 0.6977434 0.8058112 0.9063048 0.6146856
1994-05-12 0.5752051 0.1372542 0.7201138 0.8117841 0.9382432 0.6364617
1994-05-13 0.5603062 0.1420235 0.7049995 0.8208403 0.8932657 0.6331986
                [,7]      [,8]      [,9]     [,10]    [,11]     [,12]
1994-05-06 0.2775365 0.6182852 0.6995680 0.5493916 1.296570 0.3811203
1994-05-09 0.2446662 0.5657848 0.6753971 0.5139012 1.215119 0.3411054
1994-05-10 0.2664076 0.5615719 0.6678983 0.5107359 1.256880 0.3441856
1994-05-11 0.2507136 0.5504447 0.6485372 0.5154460 1.230700 0.3390521
1994-05-12 0.2654962 0.5897969 0.6551432 0.5227428 1.243511 0.3387848
1994-05-13 0.2681508 0.5938931 0.6582927 0.5309533 1.215013 0.3462120
             

This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)